In [21]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack
from astropy import units as u
from pykoa.koa import Koa
from tqdm.auto import tqdm
import os

In [14]:
sample_file = 'Gr8stars_GaiaeDR3_formatted.csv'
hires_obs_file = 'koa_out/all.csv'
koa_dir = 'koa_out/'

In [15]:
sample = Table.read(sample_file)
hires_obs = Table.read(hires_obs_file)

#### generate the list of targets for KOA online query:

In [18]:
koa_upload_file = 'hires_targets.csv'
with open(koa_upload_file, 'w') as f:
    f.write('object,ra,dec\n')
    for s in sample:
        if s['spectrum_hires'] == 'true':
            f.write('{0},{1:.16f},{2:.16f}\n'.format(s['primary_name'], s['ra_j2000'], s['dec_j2000']))

Now go to the Keck Online Archive and execute a search query for HIRES iodine-out science observations with a 30" match to that uploaded CSV of targets. Download all along with associated extracted data.  -- or not! That requires individually clicking through to each target in the table.

#### generate the list of observations:

Compile a list of all *maximum spectral resolution* iodine-out spectra

In [25]:
good_mask = (hires_obs['iodout']=='T') & (hires_obs['specres'] >= 60000) & (hires_obs['imagetyp'] == 'object')
#obs_to_download = hires_obs[good_mask]
obs_to_download = Table()
for s in sample:
    if s['spectrum_hires'] == 'true':
        object_mask = hires_obs['EDR3Name'] == 'Gaia EDR3 {0}'.format(s['gaiaedr3_source_id'])
        object_obs = hires_obs[good_mask & object_mask]
        max_res = np.max(object_obs['specres'])
        object_obs_maxres = object_obs[object_obs['specres'] == max_res]
        obs_to_download = vstack([obs_to_download, object_obs_maxres])

In [32]:
print(sample[sample['spectrum_hires_nobs'] >= 10])

     ra_j2000         dec_j2000     ... spectrum_hires_snr spectrum_hires_nobs
----------------- ----------------- ... ------------------ -------------------
 12.2446148226657    16.94064566139 ...  276.1575637204239                  22
 20.4963072108962  76.7102884073774 ...  417.4422115694578                  15
 23.6386033050351  68.9481359765662 ...   553.991877196769                  35
 29.7776387485117  33.2096831446174 ... 415.00843364924526                  15
 69.4205434010559  52.8936147114007 ...  382.7035406159708                  10
 75.2041639384368  -5.7536734242199 ...  631.1465756858703                  47
 98.3025934962235   5.4629245923217 ...  685.1488889285306                  39
 99.8811807081115  24.6001634821241 ...  220.5153055912446                  11
103.8277764975233  25.3756954298746 ...  237.1729326883656                  10
124.5997790343364 -12.6321714409806 ...  407.1387969722365                  24
167.7715498153977  30.4460175603138 ...  348.1407761

In [31]:
sample[sample['spectrum_hires_resolution'] >= 103000]

ra_j2000,dec_j2000,primary_name,gaiadr2_source_id,gaiaedr3_source_id,hip,hd,tic,2mass,wisea,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,gmag,bp_rp,radial_velocity,radial_velocity_error,ref_epoch,gaia_teff,gaia_teff_lower,gaia_teff_upper,gaia_radius,gaia_radius_lower,gaia_radius_upper,gaia_lum,gaia_lum_lower,gaia_lum_upper,spectrum_hires,spectrum_hires_resolution,spectrum_hires_snr,spectrum_hires_nobs
float64,float64,str29,int64,int64,int64,str7,int64,str16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str7,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,float64,float64,int64
133.1492129350178,28.3308208317972,rho Cnc,704967037090946688,704967037090946688,43587,75732,332064670,08523579+2819509,--,133.14676062083,0.0299,28.32978300036,0.0198,79.4482,0.0429,-485.681,0.04,-233.517,0.037,5.732681,1.008672,27.26,0.14,J2016.0,5306.0,5153.64990234375,5547.0,0.9463781118392944,0.8659300804138184,1.0031579732894897,0.639577329158783,0.6387168169021606,0.6404378414154053,true,229700.0,224.06918574404648,5
293.0899592077505,69.6611766125574,61 Dra,2261614264930275072,2261614264931057664,96100,185144,259237827,19322153+6939413,--,293.09759805108,0.0754,69.65345106255,0.0708,173.4939,0.0748,597.384,0.115,-1738.286,0.102,4.449041,0.994315,26.58,0.16,J2016.0,5450.5,5137.0,5924.0,0.7520942687988281,0.636670708656311,0.8466927409172058,0.4497639834880829,0.4479914605617523,0.45153650641441345,true,229700.0,180.2387305769767,3


In [34]:
obs_to_download.write('hires_observations.csv', overwrite=True)

In [37]:
len(obs_to_download)

1116

#### set up subdirectories for the data:

In [39]:
for s in sample:
    if s['spectrum_hires'] == 'true':        
        object_dir = './hires/{0}'.format(s['primary_name'])
        metatable = object_dir+'/obs.csv'
        
        try:  # make the directory
            os.mkdir(object_dir)
        except FileExistsError:
            pass
        
        # store the koaids for the corresponding observations in each subdirectory
        these_obs = obs_to_download[obs_to_download['EDR3Name'] == 'Gaia EDR3 {0}'.format(s['gaiaedr3_source_id'])]
        these_obs.write(metatable, overwrite=True) # populate it with names of observations to be copied over